# Agressive Social Media Text Processing



## Project Main code
1. Read in the dataset
1. Pre-process Dataset
1. Create Labels for classification
1. Build Model 1
1. Train Model 1
1. Display Accuracy
1. Build Model 2
1. Train Model 2
1. Display Accuracy
1. Transfer Learning

## Prepare Google Drive and Filename

In [2]:
#All library imports are here
import numpy as np
import pandas as pd
import re
import string
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import keras
import tensorflow as tf
import tensorflow.python.keras.backend as K
from tensorflow.python.client import device_lib
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')
from sklearn.metrics import make_scorer, f1_score, accuracy_score, recall_score, precision_score, classification_report, precision_recall_fscore_support
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re
from keras.layers.core import Dense, Dropout, Activation, Lambda
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikhi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
device = 'cuda' if torch.cuda.is_available()==True else 'cpu'
device = torch.device(device)
print(f'We are using device name "{device}"')

We are using device name "cuda"


In [4]:
dataset_file = 'D:/project/ece570nlp/dataSet.csv'

## Read Dataset



In [5]:
#In this section we Read the dataset from the Paper

# Data is stored in csv format

# Load CSV using Pandas

filename = dataset_file
names = ['id', 'post', 'label']
df_original = pd.read_csv(filename, names=names, encoding='UTF-8')
print(df_original.shape)

(12000, 3)


## Print the first 10 lines of the data as an example

In [6]:
first_10_rows = df_original.head(10)
print(first_10_rows)

                            id  \
0   facebook_corpus_msr_401470   
1   facebook_corpus_msr_386695   
2   facebook_corpus_msr_373389   
3   facebook_corpus_msr_917635   
4   facebook_corpus_msr_382517   
5   facebook_corpus_msr_403274   
6  facebook_corpus_msr_1723083   
7   facebook_corpus_msr_325257   
8    facebook_corpus_msr_23447   
9  facebook_corpus_msr_1477104   

                                                post label  
0  Mahmood Ghaznavi Aor ABdali ko bhol gaya ha tu...   OAG  
1  Bhai 60sal pehle desh me kya tha pehle pta kro...   CAG  
2  chutiya friday ko isliye releae krte kyoki wee...   CAG  
3                                         जय मोदीराज   CAG  
4     UPA walo ne bahot kuch kr diya tha desh k liye   CAG  
5  Pan ko Aadhar se link ki zarurat kuy hai? Supr...   CAG  
6  काकर पाथर जोड़ के मस्जिद लई बनाय।\n\nता चढ़ि मुल...   OAG  
7  Us raat tere papa k jageh mera sperm gya tha u...   OAG  
8                                       गटर के कीड़े   OAG  
9  Waise bandhu

## Read the processed data back to a dataframe

In [7]:
dataset_processed_file = 'D:/project/ece570nlp/DataSetProcessed.csv'

In [8]:
filename = dataset_processed_file
header = ['id', 'post', 'label']
df_processed = pd.read_csv(filename, names=header)
print(df_processed.shape)
print(df_processed.head(10))

(11999, 3)
                             id  \
0   facebook_corpus_msr_401470    
1   facebook_corpus_msr_386695    
2   facebook_corpus_msr_373389    
3   facebook_corpus_msr_917635    
4   facebook_corpus_msr_382517    
5   facebook_corpus_msr_403274    
6  facebook_corpus_msr_1723083    
7   facebook_corpus_msr_325257    
8    facebook_corpus_msr_23447    
9  facebook_corpus_msr_1477104    

                                                post label  
0   Mahmood Ghaznavi Aor ABdali ko bhol gaya ha t...   OAG  
1   Bhai 60sal pehle desh me kya tha pehle pta kr...   CAG  
2   chutiya friday ko isliye releae krte kyoki we...   CAG  
3                                       jai modiraj    CAG  
4    UPA walo ne bahot kuch kr diya tha desh k liye    CAG  
5   Pan ko Aadhar se link ki zarurat kuy hai? Sup...   CAG  
6   kaakar pathar jod ke masjid lai banaay. ta ch...   OAG  
7   Us raat tere papa k jageh mera sperm gya tha ...   OAG  
8                                    gater ke keede   

## Split the dataset into training and test data

In [9]:
df_processed = df_processed.iloc[: , 1:]  # Drop first column
print(df_processed.shape)

# Save without ID to csv file
file_no_id = 'D:/project/ece570nlp/fileNoID.csv'
df_processed.to_csv(file_no_id, index=False, header=False)

# Decide on a percentage split 70/30
#msk = np.random.rand(len(df)) < 0.7
#train_df = df_processed[msk]
#test_df = df_processed[~msk]

#print(train_df.shape)
#print(test_df.shape)

#print(train_df.head(10))

(11999, 2)


In [10]:
file_no_id = 'D:/project/ece570nlp/fileNoID.csv'
data_cleanedup = 'D:/project/ece570nlp/datacleanedup.csv'

In [11]:
header = ['post', 'label']
df_pretoken = pd.read_csv(data_cleanedup, header=None, names=header)
print(df_pretoken.shape)

X_pretoken = df_pretoken['post']
Y_pretoken = df_pretoken['label']

print(X_pretoken.shape)
print(Y_pretoken.shape)

tokenizer = Tokenizer(num_words=4000, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True,split=' ')
tokenizer.fit_on_texts(X_pretoken)
#print(tokenizer.word_index)

X = tokenizer.texts_to_sequences(X_pretoken)
#X = word_tokenize(str(X_pretoken))  # Using NLTK
#print(X)


sum_tot = 0
for seq in X:
    sum_tot += len(seq)

print(float(sum_tot/11634.0))

new_x = []
new_y = []

for i in range(0, len(X)):
    if len(X[i]) < 400:
        new_x.append(X[i])
        new_y.append(Y_pretoken[i])

print(len(new_x))
print(len(new_y))

X = pad_sequences(new_x)
Y = pd.get_dummies(new_y)

print(X.shape)
print(Y.shape)

#print(X)
#print(Y)

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 42)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)


X_train_tensor = torch.Tensor(X_train)
Y_train_tensor = torch.Tensor(np.array(Y_train))

X_train_tensor = X_train_tensor.to(device)
Y_train_tensor = Y_train_tensor.to(device)

X_test_tensor = torch.Tensor(X_test)
Y_test_tensor = torch.Tensor(np.array(Y_test))

X_test_tensor = X_test_tensor.to(device)
Y_test_tensor = Y_test_tensor.to(device)


(11788, 2)
(11788,)
(11788,)
22.9778236204229
11773
11773
(11773, 387)
(11773, 3)
(9418, 387)
(9418, 3)
(2355, 387)
(2355, 3)


Build the MLP Network

In [12]:

print(device_lib.list_local_devices())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Deep Neural Network "MLP": multi layer Perceptron
#with tf.device('/GPU:0'):
model = Sequential()


model.add(Dense(256, input_dim=X_train.shape[1]))

# 0.42 accuracy.
model.add(Activation('relu'))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(3))
model.add(Activation('softmax'))

# we'll use categorical xent for the loss, and RMSprop as the optimizer
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

print("Training...")
model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_split=0.2)

print("Generating test predictions...")
preds = model.predict(X_test, verbose=0)
print(preds)

# Evaluating the model
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = 32)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 14460300535861502525
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 10043260928
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1685448920583305316
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]
Num GPUs Available:  1
Training...
Epoch 1/5
236/236 [==============================] - 3s 3ms/step - loss: 53.0205 - accuracy: 0.3886 - val_loss: 3.4484 - val_accuracy: 0.4252
Epoch 2/5
236/236 [==============================] - 1s 2ms/step - loss: 4.1388 - accuracy: 0.3857 - val_loss: 1.0743 - val_accuracy: 0.4135
Epoch 3/5
236/236 [==============================] - 0s 2ms/step - loss: 1.4926 - accuracy: 0.4056 - val_loss: 1.0586 - val_accuracy: 0.4092
Epoch 4/5
236/236 [==============================] - 1s 2ms/step - loss: 1.2999 - accuracy: 0.3957 - val_loss: 1.0504 - val_ac

In [13]:
device = 'cuda' if torch.cuda.is_available()==True else 'cpu'
device = torch.device(device)
print(f'We are using device name "{device}"')

We are using device name "cuda"


In [14]:
predict = model.predict(X_test)
preds = predict
p = preds

74/74 [==============================] - 0s 699us/step


In [15]:
tmpMat = np.zeros((len(Y_test), 3), dtype=int)
#print(tmpMat.shape)
for i in range(0,len(predict)):
    if(p[i][0] > p[i][1] and p[i][0] > p[i][2]):
        tmpMat[i][0]=1
    elif(p[i][1] > p[i][0] and p[i][1] > p[i][2]):
        tmpMat[i][1]=1
    else:
        tmpMat[i][2]=1

test_y = Y_test.to_numpy()



print(precision_score(test_y, tmpMat, average='weighted'))
print(recall_score(test_y, tmpMat, average='weighted'))
print(f1_score(test_y, tmpMat, average='weighted'))  



# OF CAG
print(precision_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(recall_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(f1_score(test_y[:,0], tmpMat[:,0], average='weighted'))


# Of class NAG
print(precision_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(recall_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(f1_score(test_y[:,1], tmpMat[:,1], average='weighted'))

# Of class OAG
print(precision_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(recall_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(f1_score(test_y[:,2], tmpMat[:,2], average='weighted'))

0.22710698282227437
0.4101910828025478
0.24364117397481594
0.39406684532353814
0.4123142250530786
0.24784210492556172
0.6516020032365524
0.8072186836518047
0.7211102996344364
0.42346070887324794
0.6008492569002123
0.45806654887263404


c:\Users\nikhi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\nikhi\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [16]:
#CNN
# CNN: ConvNeuralNets

nb_filter = 250
filter_length = 3
hidden_dims = 250
nb_epoch = 2

In [17]:
from keras.layers.convolutional import Convolution1D
from keras import backend as K

print('Build model...')
model = Sequential()
model.add(Embedding(4000, 128))
# we add a Convolution1D, which will learn nb_filter
# word group filters of size filter_length:
model.add(Convolution1D(4000, 128, 
                        activation='relu'))

def max_1d(X_train):
    return K.max(X_train, axis=1)

model.add(Lambda(max_1d, output_shape=(nb_filter,)))
model.add(Dense(hidden_dims)) 
model.add(Dropout(0.2)) 
model.add(Activation('relu'))
model.add(Dense(3))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

Build model...


In [18]:
print('Train...')
model.fit(X_train, Y_train, batch_size=100, epochs=2,
          validation_split=0.2)
score, acc = model.evaluate(X_test, Y_test, batch_size=32)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Epoch 1/2
76/76 [==============================] - 209s 2s/step - loss: 0.6491 - accuracy: 0.4028 - val_loss: 0.5844 - val_accuracy: 0.4395
Epoch 2/2
74/74 [==============================] - 8s 100ms/step - loss: 0.5090 - accuracy: 0.6089
Test score: 0.5089619159698486
Test accuracy: 0.6089171767234802


In [19]:
predict = model.predict(X_test)
test_y = Y_test.to_numpy()
tmpMat = np.zeros((len(Y_test), 3), dtype=int)

for i in range(0,len(predict)):
    if(predict[i][0] > predict[i][1] and predict[i][0] > predict[i][2]):
        tmpMat[i][0]=1
    elif(predict[i][1] > predict[i][0] and predict[i][1] > predict[i][2]):
        tmpMat[i][1]=1
    else:
        tmpMat[i][2]=1
        
print(precision_score(test_y, tmpMat, average='weighted'))
print(recall_score(test_y, tmpMat, average='weighted'))
print(f1_score(test_y, tmpMat, average='weighted'))

print(precision_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(recall_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(f1_score(test_y[:,0], tmpMat[:,0], average='weighted'))

print(precision_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(recall_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(f1_score(test_y[:,1], tmpMat[:,1], average='weighted'))

print(precision_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(recall_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(f1_score(test_y[:,2], tmpMat[:,2], average='weighted'))

74/74 [==============================] - 7s 93ms/step
0.6141793967773401
0.6089171974522293
0.6088042268804645
0.6667732985186561
0.6564755838641189
0.6588718178408396
0.8335531591204847
0.8373673036093419
0.8353000622238581
0.7198157968214506
0.7239915074309978
0.7198117106650845


## LSTM

In [20]:
# LSTM Model for prediction
embed_dim = 128
lstm_out = 196
batch_size = 100

model = Sequential()
#model.add(Embedding(4000, embed_dim,input_length = X.shape[1], dropout = 0.2))
model.add(Embedding(4000, embed_dim,input_length = X.shape[1]))
#model.add(LSTM(lstm_out, dropout_U = 0.2, dropout_W = 0.2))
model.add(LSTM(lstm_out))
model.add(Dense(3,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 387, 128)          512000    
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense_5 (Dense)             (None, 3)                 591       
                                                                 
Total params: 767,391
Trainable params: 767,391
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
model.fit(X_train, Y_train, batch_size = batch_size, epochs = 2, verbose = 2)

Epoch 1/2
95/95 - 3s - loss: 0.9749 - accuracy: 0.4779 - 3s/epoch - 36ms/step
Epoch 2/2
95/95 - 2s - loss: 0.7817 - accuracy: 0.6282 - 2s/epoch - 22ms/step


In [22]:
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("Score: %.2f" % (score))
print("Validation Accuracy: %.2f" % (acc))

24/24 - 0s - loss: 0.8752 - accuracy: 0.5843 - 416ms/epoch - 17ms/step
Score: 0.88
Validation Accuracy: 0.58


In [23]:
p = model.predict(X_test)

74/74 [==============================] - 1s 8ms/step


In [24]:
tmpMat = np.zeros((len(Y_test), 3), dtype=int)

for i in range(0,len(predict)):
    if(p[i][0] > p[i][1] and p[i][0] > p[i][2]):
        tmpMat[i][0]=1
    elif(p[i][1] > p[i][0] and p[i][1] > p[i][2]):
        tmpMat[i][1]=1
    else:
        tmpMat[i][2]=1

test_y = Y_test.to_numpy()

# For LSTM model
print(precision_score(test_y, tmpMat, average='weighted'))
print(recall_score(test_y, tmpMat, average='weighted'))
print(f1_score(test_y, tmpMat, average='weighted'))

# Of Class CAG
print(precision_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(recall_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(f1_score(test_y[:,0], tmpMat[:,0], average='weighted'))

# Of class NAG
print(precision_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(recall_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(f1_score(test_y[:,1], tmpMat[:,1], average='weighted'))

# Of class OAG
print(precision_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(recall_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(f1_score(test_y[:,2], tmpMat[:,2], average='weighted'))

0.5857612574076241
0.5842887473460722
0.5845062839437897
0.6427934391732812
0.6386411889596603
0.6401736064155765
0.8256149751646873
0.8280254777070064
0.8267676338085272
0.6993076587617474
0.7019108280254777
0.7002618165366228


## Naive Bayes

In [25]:
import numpy as np
from sklearn import datasets

import torch
import torch.nn as nn
import torch.optim as optim

import torchbnn as bnn

model = nn.Sequential(
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=X_train.shape[1], out_features=4000),
    nn.ReLU(),
    bnn.BayesLinear(prior_mu=0, prior_sigma=0.1, in_features=4000, out_features=3),
)

model = model.to(device)


ce_loss = nn.CrossEntropyLoss()
kl_loss = bnn.BKLLoss(reduction='mean', last_layer_only=False)
kl_weight = 0.01

optimizer = optim.Adam(model.parameters(), lr=0.01)

kl_weight = 0.1

for step in range(3000):
    pre = model(X_train_tensor)
    ce = ce_loss(pre, Y_train_tensor)
    kl = kl_loss(model)
    cost = ce + kl_weight*kl
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
_, predicted = torch.max(pre.data, 1)
total = Y_train_tensor.size(0)
#correct = (predicted == Y_train_tensor).sum()
#print('- Accuracy: %f %%' % (100 * float(correct) / total))
print('- CE : %2.2f, KL : %2.2f' % (ce.item(), kl.item()))


- CE : 3.54, KL : 0.47


In [26]:
model.eval()
predict = model(X_test_tensor)
preds = predict
p = preds

In [27]:
tmpMat = np.zeros((len(Y_test_tensor), 3), dtype=int)
#print(tmpMat.shape)
for i in range(0,len(predict)):
    if(p[i][0] > p[i][1] and p[i][0] > p[i][2]):
        tmpMat[i][0]=1
    elif(p[i][1] > p[i][0] and p[i][1] > p[i][2]):
        tmpMat[i][1]=1
    else:
        tmpMat[i][2]=1


Y_test_tensor = Y_test_tensor.cpu().detach()
test_y = Y_test_tensor.numpy()



print(precision_score(test_y, tmpMat, average='weighted'))
print(recall_score(test_y, tmpMat, average='weighted'))
print(f1_score(test_y, tmpMat, average='weighted'))  



# OF CAG
print(precision_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(recall_score(test_y[:,0], tmpMat[:,0], average='weighted'))
print(f1_score(test_y[:,0], tmpMat[:,0], average='weighted'))


# Of class NAG
print(precision_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(recall_score(test_y[:,1], tmpMat[:,1], average='weighted'))
print(f1_score(test_y[:,1], tmpMat[:,1], average='weighted'))

# Of class OAG
print(precision_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(recall_score(test_y[:,2], tmpMat[:,2], average='weighted'))
print(f1_score(test_y[:,2], tmpMat[:,2], average='weighted'))

0.4150505020856367
0.4220806794055202
0.3529458830358359
0.5447077314024311
0.4653927813163482
0.41799338117676954
0.7518767875744102
0.7987261146496816
0.7594931836701142
0.5225647022187301
0.5800424628450106
0.5097267398257043


## SVM

In [65]:
#Import svm model
from sklearn import svm
from sklearn.feature_extraction.text import TfidfVectorizer

# Decide on a percentage split 70/30
msk = np.random.rand(len(df_pretoken)) < 0.7
train_df = df_pretoken[msk]
test_df = df_pretoken[~msk]

print(train_df.shape)
print(test_df.shape)

#print(train_df.head(10))

vectorizer = TfidfVectorizer(min_df=5, max_df=0.8, 
                            sublinear_tf=True, use_idf=True)
train_features = vectorizer.fit_transform(train_df['post'])
test_features = vectorizer.transform(test_df['post'])
print(train_features.shape)

#Create a svm Classifier
model = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
model.fit(train_features, train_df['label'])

#Predict the response for test dataset
predict = model.predict(test_features)

(8202, 2)
(3586, 2)
(8202, 4302)


In [73]:
preds = predict
p = preds

In [70]:
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics

# Model Accuracy: how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(test_df['label'],predict))

Accuracy: 0.5755716675962075


## Transfer Learning

In [77]:
from transformers import AutoModel, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('ai4bharat/indic-bert')
model = AutoModel.from_pretrained('ai4bharat/indic-bert')

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

Some weights of the model checkpoint at ai4bharat/indic-bert were not used when initializing AlbertModel: ['sop_classifier.classifier.weight', 'predictions.decoder.bias', 'predictions.dense.weight', 'predictions.LayerNorm.weight', 'predictions.bias', 'sop_classifier.classifier.bias', 'predictions.LayerNorm.bias', 'predictions.decoder.weight', 'predictions.dense.bias']
- This IS expected if you are initializing AlbertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [80]:
model.embeddings.word_embeddings = nn.Embedding(4000,128,padding_idx=0)
model.pooler = nn.Linear(in_features = 768, out_features=3)

model.train()

print(model)

#model.train()

#model(X_test)





AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(4000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
       